In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import random

# -------------------------- CONSTANTS --------------------------
COLOR_TO_CLASS = {
    (230, 25, 75): 0,      # Building
    (145, 30, 180): 1,     # Clutter
    (60, 180, 75): 2,      # Vegetation
    (245, 130, 48): 3,     # Water
    (255, 255, 255): 4,    # Background
    (0, 130, 200): 5,      # Car
}
CLASS_NAMES = ['Building', 'Clutter', 'Vegetation', 'Water', 'Background', 'Car']
NUM_CLASSES = len(CLASS_NAMES)
WATER_CLASS_ID = 3

# -------------------------- UTILS --------------------------


def decode_coloured_label(label_rgb: tf.Tensor) -> tf.Tensor:
    """Decodes an RGB label image into a single-channel integer class ID mask.

    This function maps pixel-wise RGB color values to their corresponding integer
    class IDs based on a predefined `COLOR_TO_CLASS` mapping. It is designed
    for use within TensorFlow graph operations.

    Args:
        label_rgb: A `tf.Tensor` of shape `(H, W, 3)` representing an RGB label image.
        Expected `dtype` is typically `tf.uint8` or convertible to `tf.uint8`.

    Returns:
        A `tf.Tensor` of shape `(H, W)` containing integer class IDs for each pixel.
    """
    label_rgb = tf.cast(label_rgb, tf.uint8)
    flat = tf.reshape(label_rgb, [-1, 3])
    keys = tf.constant(list(COLOR_TO_CLASS.keys()), dtype=tf.uint8)
    values = tf.constant(list(COLOR_TO_CLASS.values()), dtype=tf.int32)
    match = tf.reduce_all(tf.equal(tf.expand_dims(flat, 1), keys), axis=2)
    indices = tf.argmax(tf.cast(match, tf.int32), axis=1)
    return tf.reshape(indices, tf.shape(label_rgb)[:2])


def _load_npy(path: Union[tf.Tensor, str]) -> np.ndarray:
    """Loads a NumPy .npy file from a given path.

    This helper function is designed to be wrapped by `tf.py_function` for use
    within TensorFlow `tf.data` pipelines, allowing NumPy operations to run in
    the graph.

    Args:
        path: A string representing the file path to the .npy file, or a
        `tf.Tensor` containing the string path (which will be decoded).

    Returns:
        A `numpy.ndarray` loaded from the .npy file, cast to `np.float32`.
    """
    path = path.numpy().decode("utf-8") if isinstance(path, tf.Tensor) else path
    return np.load(path).astype(np.float32)

def load_image_paths(
    df: pd.DataFrame,
    image_dir: str,
    elev_dir: str,
    slope_dir: str,
    label_dir: str,
) -> Tuple[List[str], List[str], List[str], List[str], List[str]]:
    """Constructs full file paths for RGB images, elevation, slope, and label masks.

    Paths are generated based on tile IDs from a pandas DataFrame and specified
    base directories for each modality.

    Args:
        dataframe: A `pandas.DataFrame` expected to contain a "tile_id" column,
        where each entry is the unique identifier for an image tile.
        image_dir: The base directory path where RGB image files (e.g., `*-ortho.png`)
        are stored.
        elev_dir: The base directory path where elevation data files (e.g., `*-elev.npy`)
        are stored.
        slope_dir: The base directory path where slope data files (e.g., `*-slope.npy`)
        are stored.
        label_dir: The base directory path where label mask files (e.g., `*-label.png`)
        are stored.

    Returns:
        A tuple containing five lists:
        - `image_paths`: List of full file paths for RGB images.
        - `elev_paths`: List of full file paths for elevation data.
        - `slope_paths`: List of full file paths for slope data.
        - `label_paths`: List of full file paths for label masks.
        - `tile_ids`: List of original tile IDs from the input DataFrame.
    """
    tile_ids = df["tile_id"].tolist()
    image_paths = [os.path.join(image_dir, f"{tid}-ortho.png") for tid in tile_ids]
    elev_paths = [os.path.join(elev_dir, f"{tid}-elev.npy") for tid in tile_ids]
    slope_paths = [os.path.join(slope_dir, f"{tid}-slope.npy") for tid in tile_ids]
    label_paths = [os.path.join(label_dir, f"{tid}-label.png") for tid in tile_ids]
    return image_paths, elev_paths, slope_paths, label_paths, tile_ids


# -------------------------- AUGMENTATION FUNCS --------------------------


def augment_rgb_label(rgb, label):
    label = tf.expand_dims(label, axis=-1)
    if tf.random.uniform([]) > 0.5:
        rgb = tf.image.flip_left_right(rgb)
        label = tf.image.flip_left_right(label)
    if tf.random.uniform([]) > 0.5:
        rgb = tf.image.flip_up_down(rgb)
        label = tf.image.flip_up_down(label)
    if tf.random.uniform([]) > 0.0:
        k = tf.random.uniform([], 1, 4, dtype=tf.int32)
        rgb = tf.image.rot90(rgb, k)
        label = tf.image.rot90(label, k)
    if tf.random.uniform([]) > 0.75:
        rgb = tf.image.random_brightness(rgb, max_delta=0.025)
    if tf.random.uniform([]) > 0.75:
        rgb = tf.image.random_contrast(rgb, lower=0.97, upper=1.03)
    if tf.random.uniform([]) > 0.75:
        rgb = tf.image.random_saturation(rgb, lower=0.97, upper=1.03)
    if tf.random.uniform([]) > 0.8:
        rgb = tf.image.random_hue(rgb, max_delta=0.025)
    return rgb, tf.squeeze(label, axis=-1)

def augment_all(rgb, elev, slope, label):
    label = tf.expand_dims(label, axis=-1)
    if tf.random.uniform([]) > 0.5:
        rgb = tf.image.flip_left_right(rgb)
        elev = tf.image.flip_left_right(elev)
        slope = tf.image.flip_left_right(slope)
        label = tf.image.flip_left_right(label)
    if tf.random.uniform([]) > 0.5:
        rgb = tf.image.flip_up_down(rgb)
        elev = tf.image.flip_up_down(elev)
        slope = tf.image.flip_up_down(slope)
        label = tf.image.flip_up_down(label)
    if tf.random.uniform([]) > 0.0:
        k = tf.random.uniform([], 1, 4, dtype=tf.int32)
        rgb = tf.image.rot90(rgb, k)
        elev = tf.image.rot90(elev, k)
        slope = tf.image.rot90(slope, k)
        label = tf.image.rot90(label, k)
    if tf.random.uniform([]) > 0.75:
        rgb = tf.image.random_brightness(rgb, max_delta=0.025)
    if tf.random.uniform([]) > 0.75:
        rgb = tf.image.random_contrast(rgb, lower=0.97, upper=1.03)
    if tf.random.uniform([]) > 0.75:
        rgb = tf.image.random_saturation(rgb, lower=0.97, upper=1.03)
    if tf.random.uniform([]) > 0.8:
        rgb = tf.image.random_hue(rgb, max_delta=0.025)
    return rgb, elev, slope, tf.squeeze(label, axis=-1)


# -------------------------- CUTMIX FUNCS --------------------------

def _apply_water_cutmix_rgb_only(rgb, label, tile_size, water_class_id, cutmix_prob=0.9, patch_size_range=(64, 64)):
    """
    Applies self-CutMix for water class in RGB + label only setting.

    Args:
        rgb (tf.Tensor): RGB image tensor (H, W, 3)
        label (tf.Tensor): Integer label tensor (H, W)
        tile_size (int): Size of the square image (e.g., 256)
        water_class_id (int): Class index for 'water'
        cutmix_prob (float): Probability of CutMix being applied
        patch_size_range (tuple): Range of patch sizes to sample from

    Returns:
        Tuple: Augmented (rgb, label)
    """
    def return_original():
        return rgb, label

    def do_cutmix():
        water_mask = tf.cast(tf.equal(label, water_class_id), tf.float32)
        water_coords = tf.cast(tf.where(water_mask > 0), tf.int32)
        num_water_pixels = tf.shape(water_coords)[0]
        idx_max = tf.maximum(1, num_water_pixels)
        idx = tf.random.uniform([], 0, idx_max, dtype=tf.int32)

        center_y = tf.cond(num_water_pixels > 0,
                           lambda: water_coords[idx][0],
                           lambda: tf.constant(0, dtype=tf.int32))
        center_x = tf.cond(num_water_pixels > 0,
                           lambda: water_coords[idx][1],
                           lambda: tf.constant(0, dtype=tf.int32))

        patch_size = tf.random.uniform([], patch_size_range[0], patch_size_range[1] + 1, dtype=tf.int32)

        y1_src = tf.maximum(0, center_y - patch_size // 2)
        x1_src = tf.maximum(0, center_x - patch_size // 2)
        y2_src = tf.minimum(tile_size, y1_src + patch_size)
        x2_src = tf.minimum(tile_size, x1_src + patch_size)
        y1_src = tf.maximum(0, y2_src - patch_size)
        x1_src = tf.maximum(0, x2_src - patch_size)

        patch_height = y2_src - y1_src
        patch_width = x2_src - x1_src

        min_pixels_for_patch = (patch_size_range[0] * patch_size_range[0]) // 4

        condition = tf.logical_or(
            tf.less(num_water_pixels, min_pixels_for_patch),
            tf.logical_or(
                tf.less_equal(patch_height, 0),
                tf.less_equal(patch_width, 0)
            )
        )

        def return_early():
            return rgb, label

        def do_patch():
            rgb_patch = rgb[y1_src:y2_src, x1_src:x2_src, :]
            label_patch = label[y1_src:y2_src, x1_src:x2_src]

            patch_water_mask = tf.cast(tf.equal(label_patch, water_class_id), tf.float32)
            required_patch_pixels = tf.cast(patch_height * patch_width / 4, tf.float32)
            patch_water_mask_sum = tf.reduce_sum(patch_water_mask)

            def return_due_to_low_water():
                return rgb, label

            def paste_patch():
                dest_y_max = tf.maximum(1, tile_size - patch_height + 1)
                dest_x_max = tf.maximum(1, tile_size - patch_width + 1)
                dest_y = tf.random.uniform([], 0, dest_y_max, dtype=tf.int32)
                dest_x = tf.random.uniform([], 0, dest_x_max, dtype=tf.int32)

                indices_rgb = tf.stack(tf.meshgrid(
                    tf.range(dest_y, dest_y + patch_height),
                    tf.range(dest_x, dest_x + patch_width),
                    tf.range(3), indexing='ij'
                ), axis=-1)
                rgb_out = tf.tensor_scatter_nd_update(rgb, tf.reshape(indices_rgb, [-1, 3]), tf.reshape(rgb_patch, [-1]))

                indices_label = tf.stack(tf.meshgrid(
                    tf.range(dest_y, dest_y + patch_height),
                    tf.range(dest_x, dest_x + patch_width), indexing='ij'
                ), axis=-1)
                label_out = tf.tensor_scatter_nd_update(label, tf.reshape(indices_label, [-1, 2]), tf.reshape(label_patch, [-1]))

                return rgb_out, label_out

            return tf.cond(patch_water_mask_sum < required_patch_pixels, return_due_to_low_water, paste_patch)

        return tf.cond(condition, return_early, do_patch)

    # Outer probability check (CutMix is applied with cutmix_prob)
    return tf.cond(tf.random.uniform([]) < cutmix_prob, do_cutmix, return_original)

def _apply_water_cutmix_rgb_elev(rgb, elev, slope, label, tile_size, water_class_id, cutmix_prob=0.9, patch_size_range=(64, 64)):
    def return_original():
        return rgb, elev, slope, label

    def do_cutmix():
        water_mask = tf.cast(tf.equal(label, water_class_id), tf.float32)
        water_coords = tf.cast(tf.where(water_mask > 0), tf.int32)

        num_water_pixels = tf.shape(water_coords)[0]
        idx_max = tf.maximum(1, num_water_pixels)
        idx = tf.random.uniform([], 0, idx_max, dtype=tf.int32)

        center_y = tf.cond(num_water_pixels > 0,
                           lambda: water_coords[idx][0],
                           lambda: tf.constant(0, dtype=tf.int32))
        center_x = tf.cond(num_water_pixels > 0,
                           lambda: water_coords[idx][1],
                           lambda: tf.constant(0, dtype=tf.int32))

        patch_size = tf.random.uniform([], patch_size_range[0], patch_size_range[1] + 1, dtype=tf.int32)

        y1_src = tf.maximum(0, center_y - patch_size // 2)
        x1_src = tf.maximum(0, center_x - patch_size // 2)
        y2_src = tf.minimum(tile_size, y1_src + patch_size)
        x2_src = tf.minimum(tile_size, x1_src + patch_size)

        y1_src = tf.maximum(0, y2_src - patch_size)
        x1_src = tf.maximum(0, x2_src - patch_size)

        patch_height = y2_src - y1_src
        patch_width = x2_src - x1_src

        min_pixels_for_patch = (patch_size_range[0] * patch_size_range[0]) // 4

        condition = tf.logical_or(
            tf.less(num_water_pixels, min_pixels_for_patch),
            tf.logical_or(
                tf.less_equal(patch_height, 0),
                tf.less_equal(patch_width, 0)
            )
        )

        def return_early():
            return rgb, elev, slope, label

        def paste_patch():
            rgb_patch = rgb[y1_src:y2_src, x1_src:x2_src, :]
            elev_patch = elev[y1_src:y2_src, x1_src:x2_src, :]
            slope_patch = slope[y1_src:y2_src, x1_src:x2_src, :]
            label_patch = label[y1_src:y2_src, x1_src:x2_src]

            patch_water_mask = tf.cast(tf.equal(label_patch, water_class_id), tf.float32)
            required_patch_pixels = tf.cast(patch_height * patch_width / 4, tf.float32)
            patch_water_mask_sum = tf.reduce_sum(patch_water_mask)

            def return_due_to_low_water():
                return rgb, elev, slope, label

            def apply_scatter():
                dest_y_max = tf.maximum(1, tile_size - patch_height + 1)
                dest_x_max = tf.maximum(1, tile_size - patch_width + 1)
                dest_y = tf.random.uniform([], 0, dest_y_max, dtype=tf.int32)
                dest_x = tf.random.uniform([], 0, dest_x_max, dtype=tf.int32)

                indices_rgb = tf.stack(tf.meshgrid(
                    tf.range(dest_y, dest_y + patch_height),
                    tf.range(dest_x, dest_x + patch_width),
                    tf.range(3), indexing='ij'
                ), axis=-1)
                rgb_out = tf.tensor_scatter_nd_update(
                    rgb,
                    tf.reshape(indices_rgb, [-1, 3]),
                    tf.reshape(rgb_patch, [-1])
                )

                indices_elev = tf.stack(tf.meshgrid(
                    tf.range(dest_y, dest_y + patch_height),
                    tf.range(dest_x, dest_x + patch_width),
                    tf.range(1), indexing='ij'
                ), axis=-1)
                elev_out = tf.tensor_scatter_nd_update(
                    elev,
                    tf.reshape(indices_elev, [-1, 3]),
                    tf.reshape(elev_patch, [-1])
                )

                indices_slope = tf.stack(tf.meshgrid(
                    tf.range(dest_y, dest_y + patch_height),
                    tf.range(dest_x, dest_x + patch_width),
                    tf.range(1), indexing='ij'
                ), axis=-1)
                slope_out = tf.tensor_scatter_nd_update(
                    slope,
                    tf.reshape(indices_slope, [-1, 3]),
                    tf.reshape(slope_patch, [-1])
                )

                indices_label = tf.stack(tf.meshgrid(
                    tf.range(dest_y, dest_y + patch_height),
                    tf.range(dest_x, dest_x + patch_width), indexing='ij'
                ), axis=-1)
                label_out = tf.tensor_scatter_nd_update(
                    label,
                    tf.reshape(indices_label, [-1, 2]),
                    tf.reshape(label_patch, [-1])
                )

                return rgb_out, elev_out, slope_out, label_out

            return tf.cond(patch_water_mask_sum < required_patch_pixels, return_due_to_low_water, apply_scatter)

        return tf.cond(condition, return_early, paste_patch)

    return tf.cond(tf.random.uniform([]) < cutmix_prob, do_cutmix, return_original)


# -------------------------- PARSE FUNCS --------------------------

def parse_tile(rgb_path, label_path, tile_id, split='train', augment=False, tile_size=256):
    """
    Parses a single RGB image and its label. Applies augmentations if specified.

    Args:
        rgb_path (tf.Tensor): Path to the RGB image.
        label_path (tf.Tensor): Path to the label image.
        tile_id (tf.Tensor): The ID of the tile (currently unused in this function).
        split (str): 'train', 'val', or 'test' (influences augmentation).
        augment (bool): Whether to apply data augmentations.
        tile_size (int): Desired height and width of the output tiles.

    Returns:
        tuple: Processed rgb and label tensors.
    """
    rgb = tf.io.read_file(rgb_path)
    rgb = tf.image.decode_png(rgb, channels=3)
    rgb = tf.image.convert_image_dtype(rgb, tf.float32)

    label = tf.io.read_file(label_path)
    label = tf.image.decode_png(label, channels=3)
    label = decode_coloured_label(label)

    # Resize first to ensure fixed dimensions for augmentations
    rgb = tf.image.resize(rgb, [tile_size, tile_size])
    # Use 'nearest' for label resizing to preserve class integrity
    label = tf.image.resize(label[..., tf.newaxis], [tile_size, tile_size], method='nearest')
    label = tf.reshape(label, [tile_size, tile_size]) # Remove channel dimension
    label = tf.cast(label, tf.int32) # Ensure label is integer type

    if split == 'train' and augment:
        # Apply standard geometric and color augmentations
        rgb, label = augment_rgb_label(rgb, label)
        # Call the RGB-only cutmix function here
        rgb, label = _apply_water_cutmix_rgb_only(rgb, label, tile_size, WATER_CLASS_ID)

    # One-hot encode the label for model training
    label = tf.one_hot(label, depth=NUM_CLASSES)
    return rgb, label

def parse_elevation(rgb_path, elev_path, slope_path, label_path, tile_id,
                    split='train', augment=False, tile_size=256):
    """
    Parses RGB, elevation, slope, and label data for a single tile.
    Applies augmentations if specified.

    Args:
        rgb_path (tf.Tensor): Path to the RGB image.
        elev_path (tf.Tensor): Path to the elevation .npy file.
        slope_path (tf.Tensor): Path to the slope .npy file.
        label_path (tf.Tensor): Path to the label image.
        tile_id (tf.Tensor): The ID of the tile (currently unused in this function).
        split (str): 'train', 'val', or 'test' (influences augmentation).
        augment (bool): Whether to apply data augmentations.
        tile_size (int): Desired height and width of the output tiles.

    Returns:
        tuple: Processed input_image (concatenation of rgb, elev, slope) and label tensors.
    """
    rgb = tf.io.read_file(rgb_path)
    rgb = tf.image.decode_png(rgb, channels=3)
    rgb = tf.image.convert_image_dtype(rgb, tf.float32)

    label = tf.io.read_file(label_path)
    label = tf.image.decode_png(label, channels=3)
    label = decode_coloured_label(label)

    # Load .npy files using tf.py_function
    elev = tf.py_function(_load_npy, [elev_path], tf.float32)
    slope = tf.py_function(_load_npy, [slope_path], tf.float32)
    # Set shapes for the loaded npy data, as py_function returns dynamic shapes
    elev.set_shape([None, None]) # Height, Width
    slope.set_shape([None, None]) # Height, Width
    elev = tf.expand_dims(elev, axis=-1) # Add channel dimension
    slope = tf.expand_dims(slope, axis=-1) # Add channel dimension

    # Resize all inputs to the target tile_size.
    rgb = tf.image.resize(rgb, [tile_size, tile_size])
    elev = tf.image.resize(elev, [tile_size, tile_size])
    slope = tf.image.resize(slope, [tile_size, tile_size])
    label = tf.image.resize(label[..., tf.newaxis], [tile_size, tile_size], method='nearest')
    label = tf.reshape(label, [tile_size, tile_size]) # Remove channel dimension
    label = tf.cast(label, tf.int32) # Ensure label is integer type

    if split == 'train' and augment:
        # Apply standard geometric and color augmentations
        rgb, elev, slope, label = augment_all(rgb, elev, slope, label)
        # Call the RGB+Elev+Slope cutmix function here
        rgb, elev, slope, label = _apply_water_cutmix_rgb_elev(rgb, elev, slope, label, tile_size, WATER_CLASS_ID)

    # Concatenate RGB, elevation, and slope to form the combined input image
    input_image = tf.concat([rgb, elev, slope], axis=-1)
    # One-hot encode the label for model training
    label = tf.one_hot(label, depth=NUM_CLASSES)
    return input_image, label

# -------------------------- BUILD DATASET --------------------------
def build_tf_dataset(df, image_dir, elev_dir, slope_dir, label_dir,
                     input_type='rgb', batch_size=32, split='train',
                     augment=False, shuffle=True, tile_size=256):
    """
    Builds a TensorFlow Dataset for semantic segmentation.

    Args:
        df (pd.DataFrame): DataFrame containing 'tile_id' column.
        image_dir (str): Directory containing RGB images.
        elev_dir (str): Directory containing elevation .npy files.
        slope_dir (str): Directory containing slope .npy files.
        label_dir (str): Directory containing label images.
        input_type (str): 'rgb' or 'rgb_elev' to determine input channels.
        batch_size (int): Number of samples per batch.
        split (str): 'train', 'val', or 'test' (influences augmentation).
        augment (bool): Whether to apply data augmentations.
        shuffle (bool): Whether to shuffle the dataset.
        tile_size (int): Desired height and width of the output tiles.

    Returns:
        tf.data.Dataset: A TensorFlow dataset ready for training/evaluation.
    """
    image_paths, elev_paths, slope_paths, label_paths, tile_ids = load_image_paths(
        df, image_dir, elev_dir, slope_dir, label_dir
    )

    if input_type == 'rgb':
        # Dataset for RGB only
        dataset = tf.data.Dataset.from_tensor_slices((image_paths, label_paths, tile_ids))

        def map_fn(rgb_path, label_path, tile_id):
            return parse_tile(rgb_path, label_path, tile_id, split, augment, tile_size)

    elif input_type == 'rgb_elev':
        # Dataset for RGB, Elevation, and Slope
        dataset = tf.data.Dataset.from_tensor_slices((image_paths, elev_paths, slope_paths, label_paths, tile_ids))

        def map_fn(rgb_path, elev_path, slope_path, label_path, tile_id):
            return parse_elevation(rgb_path, elev_path, slope_path, label_path, tile_id, split, augment, tile_size)

    else:
        raise ValueError(f"Unsupported input_type: {input_type}")

    # Map the parsing function across the dataset elements
    dataset = dataset.map(map_fn, num_parallel_calls=tf.data.AUTOTUNE)

    if shuffle:
        # Shuffle the dataset with a buffer size equal to the number of images
        dataset = dataset.shuffle(buffer_size=len(image_paths), reshuffle_each_iteration=True)

    # Batch the dataset
    dataset = dataset.batch(batch_size)
    # Prefetch data to overlap data preprocessing and model execution
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

